In [189]:
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='darkgrid')
# palette = 'mako'
sns.set_theme(style="ticks", rc={"axes.spines.right": False, "axes.spines.top": False})

In [190]:
df = pd.read_pickle('../datasets/processed/data-test-analytics.pkl')
df.head()

,created_at,updated_at,deleted_at,birth_date,status,version,city,state,neighborhood,last_date_purchase,average_ticket,items_quantity,all_revenue,all_orders,recency,marketing_source
0,2017-08-15 07:05:00,2021-01-14 11:23:00,NaT,1974-07-10,active,2.31.7,Peixoto da Praia,AM,Aparecida 7ª Seção,2021-01-14 11:23:00,151.142944,10,906.857666,6,35,crm
1,2019-12-31 21:53:00,2021-01-08 11:23:00,NaT,1940-07-06,paused,3.30.12,Fernandes,RR,Santa Isabel,2021-01-08 11:23:00,236.991791,4,236.991791,1,41,organic_search
2,2019-03-07 23:46:00,2021-01-07 11:23:00,NaT,1963-03-18,active,3.28.9,Lopes,RR,Estrela,2021-01-07 11:23:00,211.955597,13,2331.511475,11,42,organic_search
3,2018-07-21 10:17:00,2021-01-10 11:23:00,NaT,1980-11-21,active,3.34.3,Campos do Campo,PE,Confisco,2021-01-10 11:23:00,204.113220,8,1224.679321,6,39,organic_search
4,2018-06-08 12:09:00,2021-01-18 11:23:00,NaT,1959-07-07,active,3.19.8,das Neves,RJ,Vila Suzana Segunda Seção,2021-01-18 11:23:00,252.940994,9,2023.527954,8,31,crm


In [191]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   created_at          10000 non-null  datetime64[ns]
 1   updated_at          10000 non-null  datetime64[ns]
 2   deleted_at          505 non-null    datetime64[ns]
 3   birth_date          10000 non-null  datetime64[ns]
 4   status              10000 non-null  category      
 5   version             10000 non-null  object        
 6   city                10000 non-null  object        
 7   state               10000 non-null  category      
 8   neighborhood        10000 non-null  object        
 9   last_date_purchase  10000 non-null  datetime64[ns]
 10  average_ticket      10000 non-null  float32       
 11  items_quantity      10000 non-null  int8          
 12  all_revenue         10000 non-null  float32       
 13  all_orders          10000 non-null  int8       

In [192]:
df.describe(include='number').T

,count,mean,std,min,25%,50%,75%,max
average_ticket,10000.0,216.894699,22.757214,131.378677,201.398853,217.019478,232.455040,303.386841
items_quantity,10000.0,8.499800,3.026040,1.000000,6.000000,8.000000,11.000000,19.000000
all_revenue,10000.0,1174.888550,763.141968,0.000000,494.873558,1172.751953,1798.475006,3225.654053
all_orders,10000.0,5.415400,3.457577,0.000000,2.000000,5.000000,8.000000,11.000000
recency,10000.0,67.192900,175.723276,1.000000,31.000000,35.000000,39.000000,1820.000000


In [193]:
df.describe(include=['object', 'category']).T

,count,unique,top,freq
status,10000,3,active,8524
version,10000,2905,4.26.7,24
city,10000,2406,Cardoso,161
state,10000,27,TO,409
neighborhood,10000,482,Vila Antena,36
marketing_source,10000,6,organic_search,3699


In [194]:
def birth_date_to_age(birth_date):
    today = datetime.date.today()
    return today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))

In [195]:
df['age'] = df.birth_date.apply(birth_date_to_age)
df.drop(columns=['birth_date'], inplace=True)
del birth_date_to_age

In [197]:
for coluna in df.select_dtypes(include=['datetime64[ns]']).columns:
    print(f'{coluna}: ')
    number = 1
    for i, item in zip(df[coluna].dt.year.value_counts(dropna=False).sort_index().index, df[coluna].dt.year.value_counts(dropna=False).sort_index()):
        if isinstance(i, float) and not np.isnan(i): i = int(i)
        end = '\t'
        
        if number == len(df[coluna].dt.year.value_counts(dropna=False).sort_index().index)//2: end = '\n'
        
        print(f'{i} - {item}', end=end)
        number += 1
        
    if coluna not in 'last_date_purchase': print('\n' + '-'*50 + '\n')

created_at: 
2016 - 1770	2017 - 2001	2018 - 1995
2019 - 2035	2020 - 1946	2021 - 253	
--------------------------------------------------

updated_at: 
2016 - 9	2017 - 33	2018 - 59
2019 - 106	2020 - 315	2021 - 9478	
--------------------------------------------------

deleted_at: 
2016 - 9	2017 - 33	2018 - 59
2019 - 106	2020 - 239	2021 - 59	nan - 9495	
--------------------------------------------------

last_date_purchase: 
2016 - 27	2017 - 75	2018 - 100
2019 - 125	2020 - 234	2021 - 9439	

In [209]:
pd.DataFrame(
    [
        df.status.groupby(df.created_at.dt.year).value_counts(), 
        df.status.groupby(df.updated_at.dt.year).value_counts(), 
        df.status.groupby(df.deleted_at.dt.year).value_counts(),
        df.status.groupby(df.last_date_purchase.dt.year).value_counts()
    ], index=['created_at', 'updated_at', 'deleted_at', 'last_date_purchase']
).T

created_at  updated_at  deleted_at  last_date_purchase
     status                                                          
2016 active          1480           0           0                   0
     paused           189           0           0                   0
     canceled         101           9           9                  27
2017 active          1743           0           0                   0
     paused           174           0           0                   0
     canceled          84          33          33                  75
2018 active          1693           0           0                   0
     paused           204           0           0                   0
     canceled          98          59          59                 100
2019 active          1736           0           0                   0
     paused           195           0           0                   0
     canceled         104         106         106                 125
2020 active          1660          66           0                  66
     paused           182          10           0                  10
     canceled         104         239         239                 158
2021 active           212        8458           0                8458
     paused            27         961           0                 961
     canceled          14          59          59                  20

In [202]:
df['dt_ano_mes'] = (
    df.last_date_purchase.dt.year*100
    + df.last_date_purchase.dt.month
)

df['dt_ano_mes'] = pd.to_datetime(df['dt_ano_mes'], format='%Y%m')

dummies = pd.get_dummies(df.status)

pd.concat([df, dummies], axis=1).groupby([
    'dt_ano_mes'
]).agg({
    'items_quantity': 'sum',
    'all_orders': 'sum',
    'active': 'sum',
    'canceled': 'sum',
    'paused': 'sum'
})

,items_quantity,all_orders,active,canceled,paused
dt_ano_mes,,,,,
2016-02-01,10,2,0,1,0
2016-03-01,7,9,0,1,0
2016-04-01,9,3,0,1,0
2016-05-01,7,3,0,1,0
2016-06-01,16,11,0,2,0
2016-07-01,11,9,0,1,0
2016-08-01,46,4,0,5,0
2016-10-01,71,57,0,8,0
2016-11-01,38,31,0,4,0
